In [ ]:
# 2. Solving a multiTFA model

Once you add database identifiers, Compartment information and membrane potential data to the model, you're ready to poupulate the model with thermodynamic constraints. multiTFA offers two different ways to solve the tMFA problem (Please refer to manuscript), 

1. Univariate type, where each group/component is allowed to vary in 95% confidence interval from respective mean. This is formulated as a mixed integer linear programming problem (MILP)

2. Multivariate, where group/component are drawn from multivariate normal distribution subjected to the associated linear constraints. This is a mixed integer quadratic constraint program (MIQCP). In general, MIQCP are computationally expensive.

## 2.1. populating the thermodynamic constraints

multiTFA model solver is a `optlang` model. By adding the thermodynamic constraints to the model, you're adding new variables and constraints to the `optlang` model. `optlang` currently, doesn't support quadratic constraints. So, we currently support `Cplex/Gurobi` solvers for MIQC problems. You have to access the MIQCP through the model's `gurobi_interface/cplex_interface`. These interfaces are accessible depending on what solver you are using to solve the optlang model. For example, if you are using `Cplex` as your default solver, `cplex_interface` will be available to solve the MIQCP. You can always change the solver using,



In [ ]:
# Changing solver to cplex
tfa_model.solver = 'cplex'

populating the model is easy, you simply use the `update` function. Now all the thermodynamic constraints are added to the model. Now the model contains thermodynamic constraints along with the mass balance constraints. To see if we have added all the constraints properly, you can check the number of constraints 

In [ ]:
print("Number of constraints before update {}".format(len(tfa_model.constraints)))
t_model.update()
print("Number of constraints after update {}".format(len(tfa_model.constraints)))
print(len(tfa_model.metabolites) + 6*(len(tfa_model.reactions) - len(tf_model.Exclude_reactions))) # To check if we have added correct number of constraints

In [ ]:
## 2.2. Solving the model

Once we are satisfied with the constraints, we can proceed to solve the problem. By default, `tfa_model` attempts to return the solution to MIQC problem. If `Gurobi/Cplex` is not available, then it proceeds to solve the MILP. The solution structure is a `pd.Dataframe` and you can access Fluxes, Gibbs energies of reactions and metabolite concentrations.

In [ ]:
solution =  tfa_model.optimize() # This case we are solving for MIQC problem
solution = tfa_model.optimize(solve_method = 'mip') # In this case we are solving for simple MIP problem using box method

You can always update the properties and they are reflected in the variable bounds. For example, you can change concentration bounds for metabolites and they are automatically changed for both MILP and MIQCP.

As mentioned before, MIQCP are not `optlang` models and are individual `Gurobi/Cplex` models. You can access/modify their respective interfaces according to their api.